In [63]:
try:
    from easypip import easyimport
except ModuleNotFoundError:
    from subprocess import run

    assert (
        run(["pip", "install", "easypip"]).returncode == 0
    ), "Could not install easypip"
    from easypip import easyimport

easyimport("swig")
easyimport("bbrl_utils>=0.5").setup()

In [36]:
from typing import List, Callable
import gymnasium as gym


In [37]:
from pystk2_gymnasium import AgentSpec
from functools import partial
from bbrl.agents.gymnasium import ParallelGymAgent, make_env

In [38]:
class MyWrapper(gym.ActionWrapper):
    def __init__(self, env, option: int):
        super().__init__(env)
        self.option = option

    def action(self, action):
        # We do nothing here
        return action


In [39]:
# 自定义 只保留连续 Observation 的 Wrapper
import gymnasium as gym
from gymnasium import spaces
from typing import Any, Dict, Tuple

from pystk2_gymnasium.definitions import ActionObservationWrapper


In [58]:
# 自定义 Wrapper 删除 observation space 的 离散特征
class OnlyContinousObservationWrapper(ActionObservationWrapper):
    """Removes discrete features from the observation space."""
    
    def __init__(self, env: gym.Env, **kwargs):
        super().__init__(env, **kwargs)
        
        # 过滤掉离散特征，只保留非离散特征
        self._observation_space = env.observation_space['continuous']

    def observation(self, observation: Dict):
        return observation

In [59]:
#: The base environment name
env_name = "supertuxkart/flattened_continuous_actions-v0"

#: Player name
player_name = "Example"

def get_wrappers() -> List[Callable[[gym.Env], gym.Wrapper]]:
    """Returns a list of additional wrappers to be applied to the base
    environment"""
    return [
        # Example of a custom wrapper
        # lambda env: MyWrapper(env, option="1"), 
        lambda env: OnlyContinousObservationWrapper(env),
    ]

In [61]:
if __name__ == "__main__":
    # Setup the environment
    make_stkenv = partial(
        make_env,
        env_name,
        wrappers=get_wrappers(),
        render_mode=None,
        autoreset=True,
        agent=AgentSpec(use_ai=False, name=player_name),
    )

    env_agent = ParallelGymAgent(make_stkenv, 1)
    # env_agent = OnlyContinousObservationWrapper(env_agent)

    env = env_agent.envs[0]

    print('action_space : \n', env.action_space)              # action_space 只有 discrete空间
    print('\nact_size : \n', env.action_space.shape[0])

    print('\nobservation_space : \n', env.observation_space)    # observation_space 有 continuous空间 和 discrete空间
    print('\nobs_size : \n', env.observation_space.shape[0])

    # print('\n(continuous) observation_space : \n', env.observation_space['continuous'])
    # print('\n(continuous) obs_size : \n', env.observation_space['continuous'].shape[0])

    # print('\n(discrete) observation_space : \n', env.observation_space['discrete'])
    # print('\n(discrete) obs_size : \n', env.observation_space['discrete'].shape[0])

action_space : 
 Box([ 0. -1.], 1.0, (2,), float32)

act_size : 
 2

observation_space : 
 Box([  0. -inf -inf -inf -inf -inf   0. -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf  -1.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf   0.   0.   0.
   0.   0.   0.   0. -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf  1. inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf], (91,), float32)

obs_size : 
 91


..:: Antarctica Rendering Engine 2.0 ::..
The path /dev/dri/ cannot be opened or is not available
The path /dev/dri/ cannot be opened or is not available
Unable to initialize SDL!: No available video device


error: XDG_RUNTIME_DIR not set in the environment.


In [64]:
try:
    from easypip import easyimport
except ModuleNotFoundError:
    from subprocess import run

    assert (
        run(["pip", "install", "easypip"]).returncode == 0
    ), "Could not install easypip"
    from easypip import easyimport

easyimport("swig")
easyimport("bbrl_utils>=0.5").setup()

import copy
import os

import torch
import torch.nn as nn
from bbrl.workspace import Workspace
from bbrl.agents import Agent, Agents, TemporalAgent, KWAgentWrapper
from bbrl_utils.algorithms import EpochBasedAlgo
from bbrl_utils.nn import build_mlp, setup_optimizer, soft_update_params
from bbrl_utils.notebook import setup_tensorboard
from omegaconf import OmegaConf
from torch.distributions import (
    Normal,
    Independent,
    TransformedDistribution,
    TanhTransform,
)
import bbrl_gymnasium  # noqa: F401

/home/chen_guanyu/deepdac/lib/python3.10/site-packages/bbrl_utils/notebook.py:46: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # noqa: F401
error: XDG_RUNTIME_DIR not set in the environment.


In [65]:
make_CPCenv = partial(
    make_env,
    "CartPoleContinuous-v1"
)

env_agent_1 = ParallelGymAgent(make_CPCenv, 1)
env_1 = env_agent_1.envs[0]

print('\naction_space : \n', env_1.action_space)
print('\nact_size : \n', env_1.action_space.shape[0])
print('\nobservation_space : \n', env_1.observation_space)
print('\nobs_size : \n', env_1.observation_space.shape[0])

Matplotlib backend: inline

action_space : 
 Box(-1.0, 1.0, (1,), float64)

act_size : 
 1

observation_space : 
 Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

obs_size : 
 4


In [ ]:
make_stkenv_full = partial(
    make_env,
    "supertuxkart/full-v0"
)

env_agent_2 = ParallelGymAgent(make_stkenv_full, 1)
env_2 = env_agent_2.envs[0]

print('action_space : ', env_2.action_space, '\n')
# print('act_size : ', env_2.action_space.shape[0], '\n')
print('observation_space : ', env_2.observation_space, '\n')
# print('obs_size : ', env_2.observation_space.shape[0], '\n')

action_space :  Dict('acceleration': Box(0.0, 1.0, (1,), float32), 'brake': Discrete(2), 'drift': Discrete(2), 'fire': Discrete(2), 'nitro': Discrete(2), 'rescue': Discrete(2), 'steer': Box(-1.0, 1.0, (1,), float32)) 

observation_space :  Dict('attachment': Discrete(10), 'attachment_time_left': Box(0.0, inf, (1,), float32), 'center_path': Box(-inf, inf, (3,), float32), 'center_path_distance': Box(-inf, inf, (1,), float32), 'distance_down_track': Box(-inf, inf, (1,), float32), 'energy': Box(0.0, inf, (1,), float32), 'front': Box(-inf, inf, (3,), float32), 'items_position': Sequence(Box(-inf, inf, (3,), float32), stack=False), 'items_type': Sequence(Discrete(7), stack=False), 'jumping': Discrete(2), 'karts_position': Sequence(Box(-inf, inf, (3,), float32), stack=False), 'max_steer_angle': Box(-1.0, 1.0, (1,), float32), 'paths_distance': Sequence(Box(0.0, inf, (2,), float32), stack=False), 'paths_end': Sequence(Box(-inf, inf, (3,), float32), stack=False), 'paths_start': Sequence(Box(-inf

..:: Antarctica Rendering Engine 2.0 ::..
The path /dev/dri/ cannot be opened or is not available
The path /dev/dri/ cannot be opened or is not available
Unable to initialize SDL!: No available video device


error: XDG_RUNTIME_DIR not set in the environment.
